In [5]:
pip install --upgrade snowflake-connector-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
import snowflake.connector

In [44]:
conn = snowflake.connector.connect(
    user= 'mnonog',
    password = 'KayaNatinToL0RD!',
    account='cejmwpt-djb91267'
    )

#https://cejmwpt-djb91267.snowflakecomputing.com

In [45]:
#cs = conn.cursor()

In [47]:
#cs.execute("YOUR SQL COMMAND")

In [48]:
cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")

In [ ]:
#cs.execute("USE WAREHOUSE different_warehouse_mg")

In [49]:
cs.execute("CREATE DATABASE IF NOT EXISTS testdb")

In [50]:
cs.execute("CREATE SCHEMA IF NOT EXISTS testschema")

In [51]:
cs.execute(
"CREATE OR REPLACE TABLE "    
"test_table(col1 integer, col2 string)")
cs.execute(
"INSERT INTO test_table(col1, col2) "
    		"VALUES (123, 'test string1'), (456, 'test string2')")


In [54]:
#cs.execute("PUT file:///tmp/data/file* @test_table_stage") and then cs.execute("COPY INTO test_table FROM @test_table_stage").

In [55]:
cs.execute('SELECT * FROM test_table')
print(cs.fetchmany(2))


[(123, 'test string1'), (456, 'test string2')]


In [ ]:
PUT file:///data/mydata.csv @%t1/United_States/California/Los_Angeles/2016/06/01/11/

Part II

In [1]:
# 如已安装可跳过对应行
%pip install --upgrade snowflake-connector-python
%pip install psycopg2-binary pandas lxml python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 792.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 34.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 34.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.148
    Uninstalling botocore-1.34.148:
      Successfully uninstalled botocore-1.34.148
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.10.2
    Uninstalling s3transfer-0.10.2:
      Successfully uninstalled s3transfer-0.10.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fo

In [19]:
import os
import snowflake.connector as sf

# === 建议用环境变量/ .env ===
SNOW_USER=os.getenv("SNOW_USER","<你的user>")
SNOW_PASSWORD=os.getenv("SNOW_PASSWORD","<你的password>")
SNOW_ACCOUNT=os.getenv("SNOW_ACCOUNT","svogymj-bxb71103")   # 只写标识，不要写 https://...
SNOW_ROLE=os.getenv("SNOW_ROLE","ACCOUNTADMIN")
WH_NAME=os.getenv("SNOW_WH","ETL_WH")
DB_NAME=os.getenv("SNOW_DB","ETL_DB")
SCHEMA_NAME=os.getenv("SNOW_SCHEMA","ETL_SCHEMA")

conn = sf.connect(user='ETHANAN2000', password='An67087833@123', account='svogymj-bxb71103')
cs = conn.cursor()

# 基础对象
cs.execute(f"CREATE WAREHOUSE IF NOT EXISTS {WH_NAME} WAREHOUSE_SIZE=SMALL AUTO_SUSPEND=60 AUTO_RESUME=TRUE")
cs.execute(f"CREATE DATABASE  IF NOT EXISTS {DB_NAME}")
cs.execute(f"CREATE SCHEMA    IF NOT EXISTS {DB_NAME}.{SCHEMA_NAME}")
cs.execute(f"USE ROLE {SNOW_ROLE}")
cs.execute(f"USE WAREHOUSE {WH_NAME}")
cs.execute(f"USE DATABASE {DB_NAME}")
cs.execute(f"USE SCHEMA {SCHEMA_NAME}")

# 统一 stage / 文件格式
cs.execute("CREATE STAGE IF NOT EXISTS purchases_stage")
cs.execute("CREATE STAGE IF NOT EXISTS invoices_stage")
cs.execute("CREATE STAGE IF NOT EXISTS supplier_stage")

cs.execute("""CREATE OR REPLACE FILE FORMAT csv_ff 
  TYPE=CSV FIELD_OPTIONALLY_ENCLOSED_BY='\"' SKIP_HEADER=1 NULL_IF=('','NULL')""")
cs.execute("""CREATE OR REPLACE FILE FORMAT xml_ff TYPE=XML STRIP_OUTER_ELEMENT=TRUE""")


In [31]:
# ---------- 0) 连接 ----------
# 建议把用户名/密码放到环境变量 SNOW_USER / SNOW_PASSWORD / SNOW_ACCOUNT
conn = sf.connect(
    user=os.getenv("SNOW_USER", "ETHANAN2000"),
    password=os.getenv("SNOW_PASSWORD", "An67087833@123"),
    account=os.getenv("SNOW_ACCOUNT", "svogymj-bxb71103")  # 只写标识，不要 https:// 前缀
)
cs = conn.cursor()


In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import snowflake.connector as sf


WH, DB, SC = "ETL_WH", "ETL_DB", "ETL_SCHEMA"
BASE_DIR = Path(r"/home/jovyan/MGTA 464 SQL/group project/SQL_FINAL_PROJECT/Data-5/Monthly PO Data").resolve()
OUT_CSV  = BASE_DIR / "combined_purchases.csv"   # 统一输出文件

# 2) 需要保留并标准化到 Snowflake 的列（目标列名）
TARGET_COLS = [
    "PurchaseOrderID",
    "PurchaseOrderLineID",
    "ReceivedOuters",
    "ExpectedUnitPricePerOuter",
    "OrderDate",
    "SupplierID",
]

# 3) 列名别名表（尽量覆盖你文件里的各种写法；不够的话再加几种）
ALIASES = {
    "PurchaseOrderID":           ["purchaseorderid", "purchase_order_id", "poid", "orderid"],
    "PurchaseOrderLineID":       ["purchaseorderlineid", "purchase_order_line_id", "polineid", "orderlineid"],
    "ReceivedOuters":            ["receivedouters", "received_outers", "receivedoutersqty", "receivedoutersquantity", "received_qty"],
    "ExpectedUnitPricePerOuter": ["expectedunitpriceperouter", "expected_unit_price_per_outer", "unitpriceperouter", "expectedprice", "unitprice"],
    "OrderDate":                 ["orderdate", "order_date", "date", "order_dt"],
    "SupplierID":                ["supplierid", "supplier_id", "vendorid", "vendor_id"],
}

def norm(name: str) -> str:
    """规范化列名：只保留字母数字并小写，便于匹配"""
    return "".join(ch.lower() for ch in name if ch.isalnum())

def pick_and_rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    """从原始 df 里按别名表挑出需要的列，并重命名为标准列名"""
    orig_to_norm = {c: norm(c) for c in df.columns}
    norm_to_orig = {v: k for k, v in orig_to_norm.items()}  # 取第一个出现的映射即可

    selected = {}
    missing  = []
    for tgt, alias_list in ALIASES.items():
        found = None
        for alias in alias_list:
            if alias in norm_to_orig:
                found = norm_to_orig[alias]
                break
        if found is None:
            missing.append(tgt)
        else:
            selected[tgt] = found

    if missing:
        raise ValueError(f"缺少必须列：{missing}; 文件列={list(df.columns)}")


    out = df[[selected[c] for c in TARGET_COLS]].copy()
    out.columns = TARGET_COLS
    return out

frames = []
csv_files = sorted(BASE_DIR.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"在 {BASE_DIR} 下没有找到 .csv 文件")

for p in csv_files:
    df = pd.read_csv(p, dtype=str, keep_default_na=False, na_values=["", "NULL"])
    df_std = pick_and_rename_columns(df)


    for c in ["ReceivedOuters", "ExpectedUnitPricePerOuter"]:
        df_std[c] = pd.to_numeric(df_std[c], errors="coerce")

    df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)


    df_std = df_std.dropna(subset=["PurchaseOrderID", "PurchaseOrderLineID", "OrderDate"])

    df_std["ReceivedOuters"] = df_std["ReceivedOuters"].fillna(0)
    df_std["ExpectedUnitPricePerOuter"] = df_std["ExpectedUnitPricePerOuter"].fillna(0)

    frames.append(df_std)

combined = pd.concat(frames, ignore_index=True)

combined.to_csv(OUT_CSV, index=False)
print(f"combine to 1：{OUT_CSV} line={len(combined):,}")

conn = sf.connect(
    user=os.getenv("SNOW_USER", "ETHANAN2000"),
    password=os.getenv("SNOW_PASSWORD", "An67087833@123"),
    account=os.getenv("SNOW_ACCOUNT", "svogymj-bxb71103") 
)
cs = conn.cursor()

WH, DB, SC = "ETL_WH", "ETL_DB", "ETL_SCHEMA"
cs.execute(f"CREATE WAREHOUSE IF NOT EXISTS {WH} WAREHOUSE_SIZE=SMALL AUTO_SUSPEND=60 AUTO_RESUME=TRUE")
cs.execute(f"CREATE DATABASE  IF NOT EXISTS {DB}")
cs.execute(f"CREATE SCHEMA    IF NOT EXISTS {DB}.{SC}")
cs.execute(f"USE WAREHOUSE {WH}")
cs.execute(f"USE DATABASE {DB}")
cs.execute(f"USE SCHEMA {SC}")

cs.execute("CREATE STAGE IF NOT EXISTS purchases_stage")
cs.execute("""
CREATE OR REPLACE FILE FORMAT csv_ff
  TYPE = CSV
  FIELD_OPTIONALLY_ENCLOSED_BY = '\"'
  PARSE_HEADER = TRUE
  NULL_IF = ('','NULL')
  TRIM_SPACE = TRUE
""")

cs.execute("""
CREATE OR REPLACE TABLE purchases_detail (
  PurchaseOrderID           STRING,
  PurchaseOrderLineID       STRING,
  ReceivedOuters            NUMBER(18,4),
  ExpectedUnitPricePerOuter NUMBER(18,4),
  OrderDate                 DATE,
  SupplierID                STRING
)
""")

cs.execute(f"PUT 'file:///{OUT_CSV.as_posix()}' @purchases_stage AUTO_COMPRESS=TRUE OVERWRITE=TRUE")

cs.execute("ALTER SESSION SET DATE_INPUT_FORMAT='AUTO'")
cs.execute("""
COPY INTO purchases_detail
FROM @purchases_stage
FILE_FORMAT=(FORMAT_NAME='csv_ff')
MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
ON_ERROR='ABORT_STATEMENT'
""")

# 10) 校验
cs.execute("SELECT COUNT(*) FROM purchases_detail")
print("Snowflake 行数:", cs.fetchone()[0])
cs.execute("SELECT * FROM purchases_detail ORDER BY OrderDate, PurchaseOrderID LIMIT 5")
for row in cs.fetchall():
    print(row)

cs.close(); conn.close()


/tmp/ipykernel_4817/3917673208.py:78: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_4817/3917673208.py:78: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_std["OrderDate"] = pd.to_datetime(df_std["OrderDate"], errors="coerce", infer_datetime_format=True)
/tmp/ipykernel_4817/3917673208.py:78: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, se

已生成本地整合文件：/home/jovyan/MGTA 464 SQL/group project/SQL_FINAL_PROJECT/Data-5/Monthly PO Data/combined_purchases.csv 行数=24,549


ProgrammingError: 100080 (22000): Number of columns in file (23) does not match that of the corresponding table (6), use file format option error_on_column_count_mismatch=false to ignore this error
  File '2020-12.csv.gz', line 3, character 1
  Row 1 starts at line 2, column "PURCHASES_DETAIL"[23]
  If you would like to continue loading when an error is encountered, use other values such as 'SKIP_FILE' or 'CONTINUE' for the ON_ERROR option. For more information on loading options, please run 'info loading_data' in a SQL client.